In [4]:
# 型別宣告
# 此為斷言宣告, 若型別不同則回傳Error
x = 3::Int64

3

In [5]:
# 型別錯誤
x = 3::Int8

TypeError: TypeError: in typeassert, expected Int8, got Int64

In [6]:
# 複合型別
# 型別名稱由大寫開頭並以駝峰式命名(ex: OneTwoThree) 
struct Time
    hour::Int64
    minute::Int64
    second::Int64
end

In [7]:
time1 = Time(10, 5, 0)

Time(10, 5, 0)

In [9]:
# <物件>.<欄位>
time1.hour

10

In [10]:
# 用struct建構的型別式不可變的
time1.hour = 5

ErrorException: setfield! immutable struct of type Time cannot be changed

In [11]:
# 建立可變的型別
mutable struct Time2
    hour::Int64
    minute::Int64
    second::Int64
end

In [12]:
time2 = Time2(10, 5, 0)

Time2(10, 5, 0)

In [13]:
# 改變型別欄位
time2.hour = 5

5

In [14]:
# 型別錯誤
time2.hour = "Hi"

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Int64
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Integer}, !Matched::Ptr) where T<:Integer at pointer.jl:23
  ...

In [15]:
mutable struct Time3
    hour::Int64
    minute::Int64
    second::Int64
    Time3(h, m) = new(h, m, 0)  # 內部建構子
end

# 內部建構子是一個函式, 函式名稱必須與型別名稱相同
# new只能在內部建構子中使用, 參數與型別欄位依序對應
# 此例為: 只允許變更小時及分鐘欄位, 秒數固定為0

In [17]:
time3 = Time3(3, 4)

Time3(3, 4, 0)

In [18]:
# 須遵循內部建構子的設定
time3 = Time3(3, 4, 3)

MethodError: MethodError: no method matching Time3(::Int64, ::Int64, ::Int64)
Closest candidates are:
  Time3(::Any, ::Any) at In[15]:5

In [11]:
# 參數化型別, 此例的Int64為輸入的參數
Matrix{Int64}(undef, 8, 8)

8×8 Array{Int64,2}:
  2  20  45  74   87  105  143  159
  5  23  49  76   91  120  149  162
  8  26  54  77   97  121  150  163
  9  27  56  79   98  125  152  167
 11  30  61  80   99  130  153  173
 16  35  63  82  100  132  155  174
 17  36  67  83  102  137  156  175
 19  44  73  86  103  139  158  178

In [19]:
# 參數名稱建議大寫的單一字母
struct Box{T}
    element::T
end

In [20]:
x = Box{Char}('A')
typeof(x.element)

Char

In [26]:
mutable struct Time2
    hour::Int64
    minute::Int64
    second::Int64
end

format(time::Time2) = "$(time.hour):$(time.minute):$(time.second)"

format (generic function with 1 method)

In [27]:
time = Time2(12, 0, 0)
formated_time = format(time)

"12:0:0"

In [29]:
# 判斷是否為上午或下午的方法
function isam(time::Time2)
    if 0 <= time.hour < 12
        return true
    end
    return false
end

function ispm(time::Time2)
    if 12 <= time.hour < 24
        return true
    end
    return false
end

ispm (generic function with 1 method)

In [30]:
isam(time)

false

In [31]:
ispm(time)

true